# MA5701 Optimización no lineal: Tarea 3
**Fecha de entrega:** 4 de Julio a las 23:59 hrs.

**Profesor:** Jorge G. Amaya A. 

**Auxiliar:** Aldo Gutiérrez Concha. 

**Ayudantes:** Carolina Chiu y Mariano Vazquez.

**Autor:** Felipe Urrutia Vargas

In [68]:
from scipy.optimize import linprog
import numpy as np
!pip install numdifftools
import numdifftools as nd
import re
import pandas as pd
import time

#  Método de direcciones admisibles (Zoutendijk)

Resolver el problema de optimización no lineal:
$$(P) \quad \begin{cases} \min &f (x) \\
s.a. &Ax \leq b \\
&Ex = e
\end{cases}$$

## Paso 0

In [80]:
def step0(xk, A, b, E=None, e=None):
    """
    Funcion que evalua la factibilidad del punto xk 
    en el poliedro definido por {x : Ax<=b, Ex=e}.
    
    Parametros
    ----------
    xk : np.array (n, 1)
        Punto a evaluar
    A : np.array (m_A, n)
        Coef. de las restr. de desigualdad
    b : np.array (m_A, 1)
        Lado derecho de las restr. de desigualdad
    E : default=None; np.array (m_E, n)
        Coef. de las restr. de igualdad
    e : default=None; np.array (m_E, 1)
        Lado derecho de las restr. de igualdad
    """
    feasible = np.all(A@xk <= b)
    if str(E) != "None":
        feasible = feasible and np.all(np.isclose(E@xk, e))
    return feasible

## Paso 1

In [100]:
def step1(xk, A, b, verbose=False, k=-1):
    """
    Funcion que descompone las restricciones de 
    desigualdad (Ax<=b) en aquellas que son activas 
    (A1 x = b1) y las que NO (A2 x < b2), donde 
    A = [A1 | A2] y b = [b1 | b2]
    
    Parametros
    ----------
    xk : np.array (n, 1)
        Punto a evaluar
    A : np.array (m_A, n)
        Coef. de las restr. de desigualdad
    b : np.array (m_A, 1)
        Lado derecho de las restr. de desigualdad
    """
    index_A1 = np.isclose(A@xk, b)
    index_A2 = ~index_A1

    A1 = A[index_A1]
    A2 = A[index_A2]

    b1 = b[index_A1]
    b2 = b[index_A2]
    
    if verbose:
        print(f"(step=1, k={k}) A1.index: {index_A1}")
        print(f"(step=1, k={k}) A2.index: {index_A2}")
    return A1, A2, b1, b2

## Paso 2

In [101]:
def step2(obj, A1, E, eps, method="simplex", verbose=False, k=-1):
    """
    Funcion que resuelve el problema lineal (Dk)
    definido por:
    (Dk) |  min <obj, d>
         |  s.a A1d <= 0
         |       Ed  = 0
    con dk la solucion optima.     
         
    Parametros
    ----------
    obj : np.array (n, 1)
        Costos de la funcion lineal objetivo (grad f(xk))
    A1 : np.array (m_A1, n)
        Coef. de las restr. de desigualdad activas
    E : np.array (m_E, n)
        Coef. de las restr. de igualdad
    eps : float
        Tolerancia para chequear que |obj * dk| sea pequeño
    """
    lhs_ineq = A1
    rhs_ineq = np.zeros(A1.shape[0]) if str(A1) != "None" else None

    lhs_eq = E
    rhs_eq = np.zeros(E.shape[0]) if str(E) != "None" else None

    bnd = [(-1, 1) for _ in range(obj.shape[0])] 

    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
                  A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
                  method=method)
    dk = opt.x
    stop = abs(np.dot(obj, dk))<eps
    if verbose:
        print(f"(step=2, k={k}) dk: {dk}")
    return dk, stop

## Paso 3

In [131]:
def step3(f, xk, dk, A2, b2, eps, N=10**2, sgm=0.1, verbose=False, k=-1):
    """
    Funcion que resuelve el problema lineal (Dk)
    definido por:
    (L) |  min <obj, d>
         |  s.a A1d <= 0
         |       Ed  = 0
    con dk la solucion optima.     
         
    Parametros
    ----------
    f : function
        Funcion objetivo
    xk : np.array (n, 1)
        Punto actual
    dk : np.array (n, 1)
        Direccion de descenso
    A2 : np.array (m_A2, n)
        Coef. de las restr. de desigualdad in-activas
    b2 : np.array (m_A2, 1)
        Lado derecho de las restr. de desigualdad activas
    eps : float
        Tolerancia para chequear que lmbdk sea pequeño
    N : int
        Tamaño de la discretizacion equiespaciada del intervalor 
        [0, bar_lmbdk]
    sgm : float en (0, 1)
        Pendiente del paso de armijo
    """
    # Lambda maximo
    dom = A2@dk>0
    bar_lmbdk = np.min((b2-A2@xk)[dom]/(A2@dk)[dom]) if np.sum(dom) != 0 else float("inf")

    # Paso de armijo
    ## Discretizacion del intervalo [0, bar_lmbdk]
    t = np.linspace(0, np.min([bar_lmbdk, 1]), N)
    h = 1/(N-1)
    ## Gradiente de phi en cero
    d_phi_0 = nd.Gradient(lambda t: f( xk + dk*t ))([0])
    ## lmbdk optimo
    dom = np.where(f( (xk + dk * t[:, None]).T ) <= f(xk) + sgm*d_phi_0*t)[0]
    ix_max = np.max(dom)
    lmbdk = ix_max * h * bar_lmbdk
    
    # Nuevo paso
    xk1 = xk + lmbdk * dk
    stop = lmbdk<eps
    if verbose:
        print(f"(step=3, k={k}) bar_lmbdk: {bar_lmbdk}")
        print(f"(step=3, k={k}) lmbdk: {lmbdk}")
    return xk1, stop

## Clase Zoutendijk

In [157]:
class Zoutendijk(object):
    
    def __init__(self, fun: str, const: list):
        self.fun = fun
        self.const = const
        
        self.set_vars()
        self.set_f()
        self.set_A_b_E_e()
        print(f"""-------------- Model Size --------------
   Variables    :  {self.n}
   Constraints  :  {self.m}
   Eq. consts.  :  {self.m_E}
   Ineq. consts.:  {self.m_A}
 ------------------------------------------""")
    
    def set_vars(self):
        self.vars = set(re.findall("x\d", self.fun))
        for c in self.const:
            self.vars = self.vars.union(set(re.findall("x\d", c)))   
        self.vars = sorted(list(self.vars))
        self.dic_vars = {var: f"x[{k}]" for k, var in enumerate(self.vars)}
    
    def set_f(self):
        self.rename_fun = self.fun
        for k, v in self.dic_vars.items():
            self.rename_fun = self.rename_fun.replace(k, v)
    
    def set_A_b_E_e(self):
        self.n = len(self.dic_vars)
        self.m = len(self.const)
        self.m_A = sum(int("<" in c or ">" in c) for c in self.const)
        self.m_E = self.m-self.m_A

        self.A = None
        self.E = None

        self.b = None
        self.e = None

        if self.m_A:
            self.A = np.zeros((self.m_A, self.n))
            self.b = np.zeros(self.m_A)
        if self.m_E:
            self.E = np.zeros((self.m_E, self.n))
            self.e = np.zeros(self.m_E)

        i_A = 0
        i_E = 0
        for c in self.const:
            if "<" in c or ">" in c:
                sgn = 1 if "<" in c else -1
                left, right = c.split("<=") if sgn>0 else c.split(">=")
                self.b[i_A] = sgn*float(right.strip())

                t_2 = ""
                t_1 = ""
                t = ""
                for t1 in left.strip().split():
                    if t1 in self.dic_vars.keys():
                        v = self.dic_vars[t1]
                        j = int(re.findall("\d", v)[0])

                        if t_2 == t_1 == t == "":
                            self.A[i_A, j] = sgn
                        elif t_2 == t_1 == "" and t == "-":
                            self.A[i_A, j] = sgn*(-1)
                        elif t_2 in ["", "+"] and t == "*":
                            self.A[i_A, j] = sgn*float(t_1)
                        elif t_2 == "-" and t == "*":
                            self.A[i_A, j] = sgn*float(t_1)*(-1)
                        elif t_1 in self.dic_vars.keys() and t == "+":
                            self.A[i_A, j] = sgn
                        elif t_1 in self.dic_vars.keys() and t == "-":
                            self.A[i_A, j] = sgn*(-1)
                    t_2 = t_1
                    t_1 = t
                    t = t1
                i_A+=1
            else:
                left, right = c.split("=")
                self.e[i_E] = float(right.strip())

                t_2 = ""
                t_1 = ""
                t = ""
                for t1 in left.strip().split():
                    if t1 in self.dic_vars.keys():
                        v = self.dic_vars[t1]
                        j = int(re.findall("\d", v)[0])

                        if t_2 == t_1 == t == "":
                            self.E[i_E, j] = 1
                        elif t_2 == t_1 == "" and t == "-":
                            self.E[i_E, j] = -1
                        elif t_2 in ["", "+"] and t == "*":
                            self.E[i_E, j] = float(t_1)
                        elif t_2 == "-" and t == "*":
                            self.E[i_E, j] = float(t_1)*(-1)
                        elif t_1 in self.dic_vars.keys() and t == "+":
                            self.E[i_E, j] = 1
                        elif t_1 in self.dic_vars.keys() and t == "-":
                            self.E[i_E, j] = -1
                    t_2 = t_1
                    t_1 = t
                    t = t1
                i_E+=1
                
    def f(self, x):
        return eval(self.rename_fun)
                
    def solve(self, x0, eps=1e-6, N=10**2, sgm=0.5, verbose=True):
        start=time.time()
        # Step 0: Inicializar
        k = 0
        xk = x0
        grad = nd.Gradient(self.f)
        # Step 0: Check factibilidad
        feasible = step0(xk, self.A, self.b, self.E, self.e)
        stop = not feasible
        # Step 0: Imprimir estado
        f_xk = self.f(xk)
        if verbose:
            print(f"(step=0, k={k}) Factible: {feasible}")
            print(f"(step=0, k={k}) xk: {xk}")
            print(f"(step=0, k={k}) f(xk): {f_xk}\n")
        self.list_xk = [xk]
        self.list_f_xk = [f_xk]   
        self.list_feasible = [feasible]
        while not stop:
            # Step 1: Descomposicion [A1, A2]
            A1, A2, b1, b2 = step1(xk, self.A, self.b, verbose=verbose, k=k)
            # Step 2: Resolver problema lineal (Dk)
            obj = grad(xk)
            dk, stop_grad = step2(obj, A1, self.E, eps, verbose=verbose, k=k)
            stop = stop_grad
            if not stop:
                # Step 3: Resolver problema (L)
                xk1, stop_lambda = step3(self.f, xk, dk, A2, b2, eps=eps, N=N, sgm=sgm, verbose=verbose, k=k)
                xk = xk1
                feasible = step0(xk, self.A, self.b, self.E, self.e)
                stop = (not feasible) or stop_lambda
                # Step 3: Imprimir estado
                f_xk = self.f(xk)
                if verbose:
                    print(f"(step=3, k={k}) Factible: {feasible}")
                    print(f"(step=3, k={k}) xk: {xk}")
                    print(f"(step=3, k={k}) f(xk): {f_xk}\n")
                k+=1
                self.list_xk.append(xk)
                self.list_f_xk.append(f_xk)
                self.list_feasible.append(feasible)
        if k>0:
            print(f""" ---------------------------------------------------
 Solver         :  Zoutendijk (Simplex/Armijo)
 Solution time  :  {time.time()-start:.2f} ms
 Objective      :  {self.list_f_xk[-1]: .4f}
 Stop criterion :  {"f(xk) * dk < eps" if stop_grad else "lmbdk < eps" if stop_lambda else "Unfeasible" if not feasible else "?"}
 {"Successful solution" if feasible else "?"}
 ---------------------------------------------------""")
        if k==0:
            print(f""" ---------------------------------------------------
 Solver         :  Zoutendijk (Simplex/Armijo)
 Objective      :  {self.list_f_xk[-1]: .4f}
 Start point    :  {"Unfeasible" if not feasible else "?"}
 ---------------------------------------------------""")
                
    def report(self, feasible=False):
        df = pd.DataFrame(self.list_xk, columns=self.dic_vars.keys(), index=pd.Series(range(len(self.list_xk)), name="Iter. k"))
        df["f(xk)"] = self.list_f_xk
        if feasible:
            df["Factible"] = self.list_feasible
        return df

## Problema 1

$$(P_1) \quad \begin{cases} \min &8(x_1-6)^2+(x_2-2)^4 \\
s.a. &-x_1+2x_2 \leq 4 \\
&3x_1+2x_2 \leq 12 \\
&x_i \geq 0, \quad i=1,2 \\
\end{cases}$$

In [148]:
%%time
# Inicializar datos del problema
problem_1 = Zoutendijk(
    fun="8 * ( x1 - 6 ) ** 2 + ( x2 - 2 ) ** 4", 
    const=[
        "- x1 + 2 * x2 <= 4",
        "3 * x1 + 2 * x2 <= 12",
        "x1 >= 0",
        "x2 >= 0"
    ]
)
# Resolver problema
problem_1.solve(
    x0=[0, 2], 
    eps=1e-6, 
    N=10**4, 
    sgm=0.6, 
    verbose=False
)
# Reportar resultados
problem_1.report(feasible=True)

 4
 12
 0
 0
-------------- Model Size --------------
   Variables    :  2
   Constraints  :  4
   Eq. consts.  :  0
   Ineq. consts.:  4
 ------------------------------------------
 ---------------------------------------------------
 Solver         :  Zoutendijk (Simplex/Armijo)
 Solution time  :  0.12 ms
 Objective      :   46.9029
 Stop criterion :  lmbdk < eps
 Successful solution
 ---------------------------------------------------
Wall time: 116 ms


,x1,x2,f(xk),Factible
Iter. k,,,,
0,0.000000,2.000000,288.000000,True
1,2.000000,3.000000,129.000000,True
2,4.000000,0.000000,48.000000,True
3,3.675368,0.486949,48.472334,True
4,3.830882,0.253677,46.940912,True
5,3.853022,0.220467,46.904344,True
6,3.857255,0.214117,46.902970,True
7,3.858097,0.212854,46.902915,True
8,3.858268,0.212598,46.902912,True


https://www.wolframalpha.com/input?i=minimize+8%28x+%E2%88%92+6%29%5E2+%2B+%28y+%E2%88%92+2%29%5E4+subject+to+%E2%88%92x+%2B+2y+%3C%3D+4%2C++3x+%2B+2y+%3C%3D+12%2C+x+%3E%3D+0%2C+y+%3E%3D0

## Problema 2

$$(P_2) \quad \begin{cases} \min &x_1^4-2x_2^2+10x_1x_2^2+x_4^2 \\
s.a. &x_1+x_2-x_3 = 1 \\
&x_1+x_4 = 4 \\
&x_1-x_2=0 \\
&x_i \geq 0, \quad i=1,2 \\
\end{cases}$$

In [175]:
%%time
# Inicializar datos del problema
problem_2 = Zoutendijk(
    fun="x1 ** 4 - 2 * x2 ** 2 + 10 * x1 * ( x2 ** 2 ) + x4 ** 2", 
    const=[
        "x1 + x2 - x3 = 1",
        "x1 + x4 = 4",
        "x1 - x2 = 0",
        "x1 >= 0",
        "x2 >= 0",
        "x3 >= 0",
        "x4 >= 0"
    ]
)
# Resolver problema
problem_2.solve(
    x0=[2, 2, 3, 2], 
    eps=1e-6, 
    N=10**4, 
    sgm=0.6, 
    verbose=False
)
# Reportar resultados
problem_2.report(feasible=True)

-------------- Model Size --------------
   Variables    :  4
   Constraints  :  7
   Eq. consts.  :  3
   Ineq. consts.:  4
 ------------------------------------------
 ---------------------------------------------------
 Solver         :  Zoutendijk (Simplex/Armijo)
 Solution time  :  0.17 ms
 Objective      :   13.0468
 Stop criterion :  lmbdk < eps
 Successful solution
 ---------------------------------------------------
Wall time: 169 ms


,x1,x2,x3,x4,f(xk),Factible
Iter. k,,,,,,
0,2.000000,2.000000,3.000000,2.000000,92.000000,True
1,0.500000,0.500000,0.000000,3.500000,13.062500,True
2,0.677118,0.677118,0.354235,3.322882,13.439288,True
3,0.568038,0.568038,0.136075,3.431962,13.070015,True
4,0.539051,0.539051,0.078102,3.460949,13.047803,True
5,0.532740,0.532740,0.065479,3.467260,13.046797,True
6,0.531453,0.531453,0.062905,3.468547,13.046755,True
7,0.531195,0.531195,0.062390,3.468805,13.046754,True
8,0.531145,0.531145,0.062290,3.468855,13.046754,True


https://www.wolframalpha.com/input?i=minimize+x%5E4+-+2*y%5E2%2B10*x*y%5E2%2Bw%5E2%2B0.0000001*z+subject+to+x+%2B+y-+z%3D+1%2C+x%2B+w%3D+4%2C+x-+y%3D+0%2C+x%3E%3D0%2C+y%3E%3D0%2C+z%3E%3D0%2C+w%3E%3D0

BONUS

In [174]:
%%time
# Inicializar datos del problema
problem_3 = Zoutendijk(
    fun="( 1 - x1 ) ** 2 + 100 * ( x2 - x1 ** 2) ** 2", 
    const=[
        "x1 <= 5",
        "x1 >= -5",
        "x2 <= 5",
        "x2 >= -5"
    ]
)
# Resolver problema
problem_3.solve(
    x0=[0, 2], 
    eps=1e-9, 
    N=10**4, 
    sgm=0.8, 
    verbose=True
)
# Reportar resultados
problem_3.report(feasible=True)

-------------- Model Size --------------
   Variables    :  2
   Constraints  :  4
   Eq. consts.  :  0
   Ineq. consts.:  4
 ------------------------------------------
(step=0, k=0) Factible: True
(step=0, k=0) xk: [0, 2]
(step=0, k=0) f(xk): 401

(step=1, k=0) A1.index: [False False False False]
(step=1, k=0) A2.index: [ True  True  True  True]
(step=2, k=0) dk: [ 1. -1.]
(step=3, k=0) bar_lmbdk: 5.0
(step=3, k=0) lmbdk: 5.0
(step=3, k=0) Factible: True
(step=3, k=0) xk: [ 5. -3.]
(step=3, k=0) f(xk): 78416.0

(step=1, k=1) A1.index: [ True False False False]
(step=1, k=1) A2.index: [False  True  True  True]
(step=2, k=1) dk: [-1.  1.]
(step=3, k=1) bar_lmbdk: 8.0
(step=3, k=1) lmbdk: 6.132613261326132
(step=3, k=1) Factible: True
(step=3, k=1) xk: [-1.13261326  3.13261326]
(step=3, k=1) f(xk): 346.72421409383793

(step=1, k=2) A1.index: [False False False False]
(step=1, k=2) A2.index: [ True  True  True  True]
(step=2, k=2) dk: [-1. -1.]
(step=3, k=2) bar_lmbdk: 3.8673867386738676


(step=2, k=32) dk: [-1. -1.]
(step=3, k=32) bar_lmbdk: 3.431417302976934
(step=3, k=32) lmbdk: 0.005147640718537255
(step=3, k=32) Factible: True
(step=3, k=32) xk: [-1.57373034  2.47759419]
(step=3, k=32) f(xk): 6.624181362579328

(step=1, k=33) A1.index: [False False False False]
(step=1, k=33) A2.index: [ True  True  True  True]
(step=2, k=33) dk: [ 1. -1.]
(step=3, k=33) bar_lmbdk: 6.573730337741603
(step=3, k=33) lmbdk: 0.01314877555303851
(step=3, k=33) Factible: True
(step=3, k=33) xk: [-1.56058156  2.46444541]
(step=3, k=33) f(xk): 6.640855509433615

(step=1, k=34) A1.index: [False False False False]
(step=1, k=34) A2.index: [ True  True  True  True]
(step=2, k=34) dk: [-1. -1.]
(step=3, k=34) bar_lmbdk: 3.439418437811435
(step=3, k=34) lmbdk: 0.007567477310916248
(step=3, k=34) Factible: True
(step=3, k=34) xk: [-1.56814904  2.45687793]
(step=3, k=34) f(xk): 6.595879436369294

(step=1, k=35) A1.index: [False False False False]
(step=1, k=35) A2.index: [ True  True  True  True]

(step=2, k=65) dk: [1. 1.]
(step=3, k=65) bar_lmbdk: 2.7326657837140123
(step=3, k=65) lmbdk: 0.0021863512620974197
(step=3, k=65) Factible: True
(step=3, k=65) xk: [-1.50438461  2.26952057]
(step=3, k=65) f(xk): 6.275971363905796

(step=1, k=66) A1.index: [False False False False]
(step=1, k=66) A2.index: [ True  True  True  True]
(step=2, k=66) dk: [ 1. -1.]
(step=3, k=66) bar_lmbdk: 6.504384607728076
(step=3, k=66) lmbdk: 0.00780604213348704
(step=3, k=66) Factible: True
(step=3, k=66) xk: [-1.49657857  2.26171453]
(step=3, k=66) f(xk): 6.281159980916618

(step=1, k=67) A1.index: [False False False False]
(step=1, k=67) A2.index: [ True  True  True  True]
(step=2, k=67) dk: [-1. -1.]
(step=3, k=67) bar_lmbdk: 3.5034214344054107
(step=3, k=67) lmbdk: 0.005255657717379855
(step=3, k=67) Factible: True
(step=3, k=67) xk: [-1.50183422  2.25645887]
(step=3, k=67) f(xk): 6.259265270081181

(step=1, k=68) A1.index: [False False False False]
(step=1, k=68) A2.index: [ True  True  True  True

(step=2, k=94) dk: [-1. -1.]
(step=3, k=94) bar_lmbdk: 3.5728544279613885
(step=3, k=94) lmbdk: 0.009290350547754386
(step=3, k=94) Factible: True
(step=3, k=94) xk: [-1.43643592  2.05908432]
(step=3, k=94) f(xk): 5.938038041996591

(step=1, k=95) A1.index: [False False False False]
(step=1, k=95) A2.index: [ True  True  True  True]
(step=2, k=95) dk: [1. 1.]
(step=3, k=95) bar_lmbdk: 2.9409156849621123
(step=3, k=95) lmbdk: 0.0029412098059427064
(step=3, k=95) Factible: True
(step=3, k=95) xk: [-1.43349471  2.06202552]
(step=3, k=95) f(xk): 5.926963726358485

(step=1, k=96) A1.index: [False False False False]
(step=1, k=96) A2.index: [ True  True  True  True]
(step=2, k=96) dk: [ 1. -1.]
(step=3, k=96) bar_lmbdk: 6.433494712780423
(step=3, k=96) lmbdk: 0.007720965751911699
(step=3, k=96) Factible: True
(step=3, k=96) xk: [-1.42577375  2.05430456]
(step=3, k=96) f(xk): 5.930490600432017

(step=1, k=97) A1.index: [False False False False]
(step=1, k=97) A2.index: [ True  True  True  Tru

(step=1, k=127) A1.index: [False False False False]
(step=1, k=127) A2.index: [ True  True  True  True]
(step=2, k=127) dk: [-1. -1.]
(step=3, k=127) bar_lmbdk: 3.658673901795154
(step=3, k=127) lmbdk: 0.010977119417327196
(step=3, k=127) Factible: True
(step=3, k=127) xk: [-1.35230322  1.82209536]
(step=3, k=127) f(xk): 5.5377243005113455

(step=1, k=128) A1.index: [False False False False]
(step=1, k=128) A2.index: [ True  True  True  True]
(step=2, k=128) dk: [1. 1.]
(step=3, k=128) bar_lmbdk: 3.1779046370867587
(step=3, k=128) lmbdk: 0.0041316891971325
(step=3, k=128) Factible: True
(step=3, k=128) xk: [-1.34817153  1.82622705]
(step=3, k=128) f(xk): 5.5214100950582

(step=1, k=129) A1.index: [False False False False]
(step=1, k=129) A2.index: [ True  True  True  True]
(step=2, k=129) dk: [ 1. -1.]
(step=3, k=129) bar_lmbdk: 6.34817152842504
(step=3, k=129) lmbdk: 0.007618567690879137
(step=3, k=129) Factible: True
(step=3, k=129) xk: [-1.34055296  1.81860848]
(step=3, k=129) f(xk)

...

(step=3, k=1906) bar_lmbdk: 3.100041500108127
(step=3, k=1906) lmbdk: 0.004030456995840149
(step=3, k=1906) Factible: True
(step=3, k=1906) xk: [-1.90398896  3.62843865]
(step=3, k=1906) f(xk): 8.434217688744619

(step=1, k=1907) A1.index: [False False False False]
(step=1, k=1907) A2.index: [ True  True  True  True]
(step=2, k=1907) dk: [ 1. -1.]
(step=3, k=1907) bar_lmbdk: 6.903988956887713
(step=3, k=1907) lmbdk: 0.006904679424830197
(step=3, k=1907) Factible: True
(step=3, k=1907) xk: [-1.89708428  3.62153397]
(step=3, k=1907) f(xk): 8.44419687781117

(step=1, k=1908) A1.index: [False False False False]
(step=1, k=1908) A2.index: [ True  True  True  True]
(step=2, k=1908) dk: [-1. -1.]
(step=3, k=1908) bar_lmbdk: 3.1029157225371167
(step=3, k=1908) lmbdk: 0.0040341938586841205
(step=3, k=1908) Factible: True
(step=3, k=1908) xk: [-1.90111847  3.61749978]
(step=3, k=1908) f(xk): 8.41754355156238

(step=1, k=1909) A1.index: [False False False False]
(step=1, k=1909) A2.index: [ True 

(step=3, k=1938) bar_lmbdk: 3.1496253654557753
(step=3, k=1938) lmbdk: 0.004409916503288414
(step=3, k=1938) Factible: True
(step=3, k=1938) xk: [-1.85478455  3.44229038]
(step=3, k=1938) f(xk): 8.150221111277792

(step=1, k=1939) A1.index: [False False False False]
(step=1, k=1939) A2.index: [ True  True  True  True]
(step=2, k=1939) dk: [ 1. -1.]
(step=3, k=1939) bar_lmbdk: 6.854784551047514
(step=3, k=1939) lmbdk: 0.008226564117668785
(step=3, k=1939) Factible: True
(step=3, k=1939) xk: [-1.84655799  3.43406382]
(step=3, k=1939) f(xk): 8.161880243178619

(step=1, k=1940) A1.index: [False False False False]
(step=1, k=1940) A2.index: [ True  True  True  True]
(step=2, k=1940) dk: [-1. -1.]
(step=3, k=1940) bar_lmbdk: 3.1534420130701557
(step=3, k=1940) lmbdk: 0.004730636083213556
(step=3, k=1940) Factible: True
(step=3, k=1940) xk: [-1.85128862  3.42933318]
(step=3, k=1940) f(xk): 8.130272662787535

(step=1, k=1941) A1.index: [False False False False]
(step=1, k=1941) A2.index: [ Tru

(step=3, k=1968) bar_lmbdk: 3.2085328278458607
(step=3, k=1968) lmbdk: 0.005775936683790929
(step=3, k=1968) Factible: True
(step=3, k=1968) xk: [-1.79724311  3.23078207]
(step=3, k=1968) f(xk): 7.8246179092444

(step=1, k=1969) A1.index: [False False False False]
(step=1, k=1969) A2.index: [ True  True  True  True]
(step=2, k=1969) dk: [ 1. -1.]
(step=3, k=1969) bar_lmbdk: 6.797243108837931
(step=3, k=1969) lmbdk: 0.010196884351692066
(step=3, k=1969) Factible: True
(step=3, k=1969) xk: [-1.78704622  3.22058519]
(step=3, k=1969) f(xk): 7.840802208398335

(step=1, k=1970) A1.index: [False False False False]
(step=1, k=1970) A2.index: [ True  True  True  True]
(step=2, k=1970) dk: [-1. -1.]
(step=3, k=1970) bar_lmbdk: 3.212953775513762
(step=3, k=1970) lmbdk: 0.005783895185443316
(step=3, k=1970) Factible: True
(step=3, k=1970) xk: [-1.79283012  3.21480129]
(step=3, k=1970) f(xk): 7.7999316005274775

(step=1, k=1971) A1.index: [False False False False]
(step=1, k=1971) A2.index: [ True 

(step=2, k=2001) dk: [-1. -1.]
(step=3, k=2001) bar_lmbdk: 3.272436946236008
(step=3, k=2001) lmbdk: 0.004581869911721583
(step=3, k=2001) Factible: True
(step=3, k=2001) xk: [-1.73214492  3.00216215]
(step=3, k=2001) f(xk): 7.464953015777554

(step=1, k=2002) A1.index: [False False False False]
(step=1, k=2002) A2.index: [ True  True  True  True]
(step=2, k=2002) dk: [ 1. -1.]
(step=3, k=2002) bar_lmbdk: 6.732144923675714
(step=3, k=2002) lmbdk: 0.010099227308244396
(step=3, k=2002) Factible: True
(step=3, k=2002) xk: [-1.7220457   2.99206292]
(step=3, k=2002) f(xk): 7.4804034329603954

(step=1, k=2003) A1.index: [False False False False]
(step=1, k=2003) A2.index: [ True  True  True  True]
(step=2, k=2003) dk: [-1. -1.]
(step=3, k=2003) bar_lmbdk: 3.277954303632531
(step=3, k=2003) lmbdk: 0.005900907837322288
(step=3, k=2003) Factible: True
(step=3, k=2003) xk: [-1.7279466   2.98616202]
(step=3, k=2003) f(xk): 7.441705819601812

(step=1, k=2004) A1.index: [False False False False]
(s

KeyboardInterrupt: 